### Get whisper to `dmx.compiler` with alternative `Conv1d` subgraph

In [3]:
model_id = "openai/whisper-tiny"

In [ ]:
# original inference

import torch
from transformers import pipeline

pipe = pipeline(
    task="automatic-speech-recognition", 
    model=model_id,
    device="cuda",
)

pipe("audio.mp3")

In [ ]:
# add these additional lines before transformation/configuration to map Conv1d to our alternative version
from dmx.compressor.modeling.nn import experimental
from dmx.compressor.fx.transformer.utils import dmx_aware_mapping

dmx_aware_mapping["torch.nn.modules.conv.Conv1d"] = experimental.Conv1d

# ---
# transformation and configuration
from dmx.compressor import DmxModel

pipe.model = DmxModel.from_torch(pipe.model)
pipe.model.to_basic_mode()

In [ ]:
pipe("audio.mp3")

In [ ]:
pipe.model.dmx_config

In [ ]:
# do a forward that triggers JIT tracing through encoder
inputs = {
    "input_features": torch.rand(1, 80, 3000).to("cuda"),
    "decoder_input_ids": torch.tensor([[50258]], device="cuda"),
}
output = pipe.model(**inputs)

# QdQ transform pipe.model._gm and show there is no conv1d in encoder
from dmx.compressor.fx.transform import make_compiler_graph
import dmir_compiler
compiler_graph = make_compiler_graph(pipe.model._gm)

In [ ]:
pipe.model._gm.graph.print_tabular()

In [ ]:
compiler_graph.graph.print_tabular()